<a href="https://colab.research.google.com/github/Kim-TaeKyoung/one/blob/main/2_DeviceCSVRefine_AlphaposeConcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
import os
import pathlib
import numpy as np
import pytz
from natsort import natsorted
from datetime import datetime
from pprint import pprint

In [ ]:
os.environ['SHARED_DIR'] = '/content/drive/MyDrive/Colab/'
os.environ['CSV_DIR'] = os.environ['SHARED_DIR'] + 'FieldTestCSV/AllCSVs/'
os.environ['VIDEO_DIR'] = os.environ['SHARED_DIR'] + 'frames/'
os.environ['ALPHAPOSE_RESULT_DIR'] = os.environ['VIDEO_DIR'] + 'Alphaposed/'
os.environ['REFINED_DIR'] = os.environ['SHARED_DIR'] + 'RefinedCSV/'

VALID_SECOND = 0.1
FRONT_SECOND = 0.1
ATTENTION1_SECOND = 0.3
ATTENTION2_SECOND = 3

In [ ]:
# 20211012 to 20211023
TARGET_DATE = '20211012'
FRAME_TARGET_DATE = '2021_10_12'

In [ ]:
df = pd.read_csv(os.environ['CSV_DIR'] + 'Device_Output-All-' + TARGET_DATE + '.csv')

In [ ]:
df

,segment_name,minutes,timestamp,ad_id,current_time,lat,lon,alt,speed,weatherstamp,illuminance
0,NaN,NaN,20211012-061253KST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,20211012-061253KST,아이디올시니어버전,NaN,NaN,NaN,NaN,NaN,NaN,14.166667
2,NaN,0.0,20211012-061255KST,아이디올시니어버전,NaN,NaN,NaN,NaN,NaN,NaN,10.833333
3,NaN,0.0,20211012-061257KST,아이디올시니어버전,NaN,0.000000,0.000000,NaN,NaN,NaN,30.416667
4,NaN,0.0,20211012-061259KST,아이디올시니어버전,NaN,0.000000,0.000000,NaN,0.00000,NaN,30.833333
...,...,...,...,...,...,...,...,...,...,...,...
10153,20211012-170633KST,4.0,20211012-172313KST,아이디올시니어버전,NaN,37.516720,127.019800,37.8,2.03720,NaN,1131.666667
10154,20211012-170633KST,5.0,20211012-172315KST,아이디올시니어버전,NaN,37.516713,127.019785,37.5,2.44464,NaN,1126.666667
10155,20211012-170633KST,5.0,20211012-172317KST,아이디올시니어버전,NaN,37.516708,127.019770,37.3,2.50020,NaN,1127.916667
10156,20211012-170633KST,5.0,20211012-172319KST,아이디올시니어버전,NaN,37.516705,127.019762,37.1,1.87052,NaN,1102.083333


In [ ]:
# gps_df = pd.read_csv(os.environ['SHARED_DIR'] + 'refined_gps.csv')
# dummy = gps_df.pop('Unnamed: 0')
# gps_df

In [ ]:
def get_datetime(time_str: str):
  time_str = datetime.strptime(time_str[:-3], "%Y%m%d-%H%M%S")
  seoul_timezone = pytz.timezone("Asia/Seoul") # hardcoded
  datetimed = seoul_timezone.localize(time_str).astimezone(seoul_timezone)

  return datetimed

In [ ]:
# segments = df['segment_name'].unique()
# segments

# for i, row in df.iterrows():
#   # Can't we just use ISO format?
#   device_time = datetime.strptime(row['timestamp'], "%Y%m%d-%H%M%S")

#   lon = row['lon']
#   lat = row['lat']

#   if lon != lon and lat != lat:
#     phone_data = gps_df[(gps_df.year == device_time.year) & 
#           (gps_df.month == device_time.month) & 
#           (gps_df.day == device_time.day) & 
#           (gps_df.hour == device_time.hour) & 
#           (gps_df.minute == device_time.minute)]
    
#     if not phone_data.empty:
#       plon = phone_data['Lon']
#       plat = phone_data['Lat']

#       df.at[i, 'lon'] = plon
#       df.at[i, 'lat'] = plat


In [ ]:
# df

In [ ]:
df['time_hour'] = -1
df['time_minute'] = -1
df['head_count'] = ''
df['T_valid_count'] = ''
df['T_front_count'] = ''
df['T_valid_second'] = ''
df['T_front_second'] = ''
df['T_attention1_count'] = ''
df['T_attention2_count'] = ''
df['T_attention1_second'] = ''
df['T_attention2_second'] = ''
df['video_file'] = ''
df

,segment_name,minutes,timestamp,ad_id,current_time,lat,lon,alt,speed,weatherstamp,illuminance,time_hour,time_minute,head_count,T_valid_count,T_front_count,T_valid_second,T_front_second,T_attention1_count,T_attention2_count,T_attention1_second,T_attention2_second,video_file
0,NaN,NaN,20211012-061253KST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,,,,,,,,,,
1,NaN,0.0,20211012-061253KST,아이디올시니어버전,NaN,NaN,NaN,NaN,NaN,NaN,14.166667,-1,-1,,,,,,,,,,
2,NaN,0.0,20211012-061255KST,아이디올시니어버전,NaN,NaN,NaN,NaN,NaN,NaN,10.833333,-1,-1,,,,,,,,,,
3,NaN,0.0,20211012-061257KST,아이디올시니어버전,NaN,0.000000,0.000000,NaN,NaN,NaN,30.416667,-1,-1,,,,,,,,,,
4,NaN,0.0,20211012-061259KST,아이디올시니어버전,NaN,0.000000,0.000000,NaN,0.00000,NaN,30.833333,-1,-1,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10153,20211012-170633KST,4.0,20211012-172313KST,아이디올시니어버전,NaN,37.516720,127.019800,37.8,2.03720,NaN,1131.666667,-1,-1,,,,,,,,,,
10154,20211012-170633KST,5.0,20211012-172315KST,아이디올시니어버전,NaN,37.516713,127.019785,37.5,2.44464,NaN,1126.666667,-1,-1,,,,,,,,,,
10155,20211012-170633KST,5.0,20211012-172317KST,아이디올시니어버전,NaN,37.516708,127.019770,37.3,2.50020,NaN,1127.916667,-1,-1,,,,,,,,,,
10156,20211012-170633KST,5.0,20211012-172319KST,아이디올시니어버전,NaN,37.516705,127.019762,37.1,1.87052,NaN,1102.083333,-1,-1,,,,,,,,,,


In [ ]:
alphapose_csvs = list(pathlib.Path(os.environ['ALPHAPOSE_RESULT_DIR'] + FRAME_TARGET_DATE).rglob('*.csv'))
alphapose_csvs_dict = {}

for path in alphapose_csvs:
  strip_filename, _ = os.path.split(str(path))
  _, segment_name = os.path.split(strip_filename)
  minute = os.path.split(str(path))[-1].split('_')[-1].split('.')[0]

  if not segment_name in alphapose_csvs_dict:
    alphapose_csvs_dict.update({segment_name : {}})
  
  if not minute in alphapose_csvs_dict[segment_name]:
    alphapose_csvs_dict[segment_name].update({minute : ''})

  alphapose_csvs_dict[segment_name][minute] = str(path)

alphapose_csv_cache = {}

for segment in alphapose_csvs_dict.keys():
  for minute in alphapose_csvs_dict[segment].keys():
    path = alphapose_csvs_dict[segment][minute]
    alphapose_csv_cache.update({path : pd.read_csv(path)})

In [ ]:
for i, row in df.iterrows():
  if row['segment_name'] != row['segment_name'] or row['minutes'] != row['minutes']:
    continue
  else:
    segment_name = row['segment_name']
    minute = str(int(row['minutes']))
    try:
      target_dir = alphapose_csvs_dict[segment_name][minute]
      alphapose_csv = alphapose_csv_cache[target_dir]

      datetimed = get_datetime(row['timestamp'])
      time_hour = datetimed.hour
      time_minute = datetimed.minute

      head_count = len(alphapose_csv)

      total_valid_count = len(alphapose_csv[alphapose_csv['T_valid'] > VALID_SECOND])
      total_front_count = len(alphapose_csv[alphapose_csv['T_front'] > FRONT_SECOND])
      total_valid_second = (alphapose_csv[alphapose_csv['T_valid'] > VALID_SECOND]['T_valid']).sum()
      total_front_second = (alphapose_csv[alphapose_csv['T_front'] > FRONT_SECOND]['T_front']).sum() 
      
      total_atc1_count = len(alphapose_csv[alphapose_csv['T_attention'] > ATTENTION1_SECOND])
      total_atc2_count = len(alphapose_csv[alphapose_csv['T_attention'] > ATTENTION2_SECOND])
      total_atc1_second = (alphapose_csv[alphapose_csv['T_attention'] > ATTENTION1_SECOND]['T_attention']).sum()
      total_atc2_second = (alphapose_csv[alphapose_csv['T_attention'] > ATTENTION2_SECOND]['T_attention']).sum()
      video_file = target_dir.replace('csv', 'mp4')

      df.at[i, 'time_hour'] = time_hour
      df.at[i, 'time_minute'] = time_minute

      df.at[i, 'head_count'] = head_count
      df.at[i, 'T_valid_count'] = total_valid_count
      df.at[i, 'T_front_count'] = total_front_count
      df.at[i, 'T_valid_second'] = total_valid_second
      df.at[i, 'T_front_second'] = total_front_second
      df.at[i, 'T_attention1_count'] = total_atc1_count
      df.at[i, 'T_attention2_count'] = total_atc2_count
      df.at[i, 'T_attention1_second'] = total_atc1_second
      df.at[i, 'T_attention2_second'] = total_atc2_second
      df.at[i, 'video_file'] = video_file
    except KeyboardInterrupt:
      break
    except KeyError as e:
      print('Missing video', segment_name, minute)



Missing video 20211012-102401KST 4
Missing video 20211012-102401KST 5
Missing video 20211012-102401KST 7
Missing video 20211012-102401KST 10
Missing video 20211012-102401KST 16
Missing video 20211012-102401KST 17
Missing video 20211012-102401KST 18
Missing video 20211012-102401KST 20
Missing video 20211012-121150KST 12
Missing video 20211012-121150KST 20
Missing video 20211012-121150KST 21
Missing video 20211012-121150KST 23
Missing video 20211012-121150KST 26
Missing video 20211012-121150KST 30
Missing video 20211012-124441KST 1
Missing video 20211012-124441KST 6
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing video 20211012-131812KST 0
Missing v

In [ ]:
df.to_csv(os.environ['REFINED_DIR'] + TARGET_DATE + '_PhoneGPS-AlphaPose-Device-Output.csv', index=False, encoding="utf-8-sig")

In [ ]:
# df